In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import torch
import kornia
import numpy as np
import gc
import os
import sys
%matplotlib notebook
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

from helpers import Trainer
from helpers import Logger
from helpers import Trial
from mibi_dataloader import MIBIData
from modules import SelfSupervisedEstimator
from criteria import SelfSupervisedEstimatorLoss
import utils

In [2]:
# Load the data
main_dir = '/home/hazmat/GitHub/Denoisotron/'
train_dir = main_dir + 'data/traindat/'
test_dir = main_dir + 'data/testdat/'

modl_dir = main_dir + 'models/'
rslt_dir = main_dir + 'results/'

# train_ds = MIBIData(folder=train_dir, crop=31, scale=1, stride=8, number=10000)
# test_ds = MIBIData(folder=test_dir, crop=31, scale=1, stride=8)
lambda_ds = MIBIData.depickle('/home/hazmat/GitHub/Denoisotron/data/large_lambda_ds')

In [3]:
# def l_est(samples, bs, ts):
#     l_ests = torch.zeros(samples.shape)
#     for i in range(samples.size(0)):
#         l_ests[i,:,:,:], cimg = utils.estimate_lambda(samples[i,0,:,:], bs, ts)
#         print('\rProcessing.......' + str(100 * i / samples.size(0)) + '%', end='')
#     return l_ests

# bs = [10];
# while bs[-1]>.7:
#     bs.append(bs[-1]*0.8)

# ts = torch.tensor(bs).float()**-2
# bs = np.array(bs)
# ts = ts.numpy()

# l_images = l_est(train_ds.images, bs, ts)

# lambda_ds = MIBIData(images=l_images, labels='None', source=train_ds.source, crop=31, scale=1, stride=8)
# lambda_ds.pickle('/home/hazmat/GitHub/Denoisotron/data/large_lambda_ds')

In [4]:
estimator_args = dict()
# estimator_args['kdims'] =  [15]
# estimator_args['nfilts'] = [0]

torch.cuda.empty_cache()
# estimator = LambdaEstimator(**estimator_args)
# estimator = LambdaEstimator.load_model(main_dir + 'estimator/models/estimator_saves/', 'model_109')
# estimator = SelfSupervisedEstimator(**estimator_args)
estimator = SelfSupervisedEstimator.load_model(main_dir + 'estimator/models/estimator_saves/', 'amazing_model')

estimator.cuda()
estimator_logger = Logger(['loss'])
estimator_trainer = Trainer()

In [5]:
# Estimator training parameters
estimator_train_args = dict()
estimator_train_args['lr'] = 0.0001
estimator_train_args['batch_size'] = 100
estimator_train_args['epochs'] = 10
estimator_train_args['report'] = 5
estimator_train_args['crop'] = 121
estimator_train_args['clip'] = 1
estimator_train_args['decay'] = 0
estimator_train_args['restart'] = False
estimator_train_args['epoch_frac'] = 1
# estimator_train_args['error_target'] = 0.4
# estimator_train_args['decay'] = 1e-5

# Estimator loss parameters
estimator_criterion = SelfSupervisedEstimatorLoss()

lambda_ds.set_crop(estimator_train_args['crop'])

# estimator.noise = torch.tensor(1)

2304


In [6]:
estimator.cuda()
estimator_train_args['continue'] = True
estimator_trainer.train(estimator, lambda_ds, estimator_criterion, estimator_logger, main_dir + 'estimator/models/', **estimator_train_args)
# print(estimator.noise)

2304
Epoch:0 > < 0.02773431102956276                                                                                                                    
Epoch:1 > < 0.02767236893478373                                                                                                                    
Epoch:2 > < 0.02764169652295472                                                                                                                    
Epoch:3 > < 0.027620545459274063                                                                                                                   
Epoch:4 > < 0.027605187734290312                                                                                                                   
Epoch:5 > < 0.027595929520206893                                                                                                                   
Epoch:6 > < 0.027582977054445214                                                                           

278825.5082011223

In [ ]:
# filt_fig = plt.figure()
# filt = estimator.filters[10].lambda_estimator.conv.weight.cpu().detach()[0,0,:,:]
# print(torch.sum(filt))
# plt.imshow(filt)

In [ ]:
estimator.cuda()
fig = plt.figure(figsize=(12,6))
data = test_ds.images[271].unsqueeze(0)
lamd = estimator.process(data.cuda())
ax1 = plt.subplot(1,2,1)
plt.imshow(data[0,0,:,:].cpu().detach())
ax2 = plt.subplot(1,2,2, sharex=ax1, sharey=ax1)
plt.imshow(lamd[0,0,:,:].cpu().detach())

In [ ]:
# AABAAAAPoint1_3_Abeta40.tif
# print(train_ds.source[1])
data = train_ds.images[2].unsqueeze(0).cuda()
l_hat = estimator.process(data)
l_hat[l_hat<0]=0
resample = torch.tensor(np.random.poisson(l_hat.cpu().detach().numpy()))

In [ ]:
# estimator.eval()
# estimator.cpu()
# msecrit = torch.nn.MSELoss()
# data = get_even_data(test_ds, label_dict).cpu()
# n_iters = 20
# r_data_1 = torch.tensor(data).cpu();
# r_data_2 = torch.tensor(data).cpu();
# error_1 = list()
# error_2 = list()
# data_hist_1 = list()
# data_hist_2 = list()
# est_hist_1 = list()
# est_hist_2 = list()

# for i in range(n_iters):
#     r_data_1 = gdt_estimator(r_data_1.cpu(), bs, ts).cpu()
#     r_data_2 = net_estimator(r_data_2, estimator).cpu()
#     est_hist_1.append(r_data_1)
#     est_hist_2.append(r_data_2)
#     # mean_val = torch.mean(r_data)
#     loss_1 = msecrit(r_data_1, data)
#     loss_2 = msecrit(r_data_2, data)
    
#     error_1.append(loss_1.item())
#     error_2.append(loss_2.item())
    
#     r_data_1 = torch.poisson(r_data_1.detach()).cpu()
#     r_data_2 = torch.poisson(r_data_2.detach()).cpu()
#     print(i)

In [ ]:
# we need to specify a model
# we should sample multiple times at each model, on each iteration, for each sample

def get_data_even(dataset, label_dict):
    avoid_label = ['Au', 'Background', 'C', 'Ca', 'Fe', 'Na', 'Si', 'Ta', 'totalIon']
    idxs = list()
    for label in label_dict.keys():
        if not any(label==avoid for avoid in avoid_label):
            idx = np.random.choice(label_dict[label])
            idxs.append(idx)
    data = dataset.images[idxs]
    return data.cuda()
            
# randomly pick some data
def get_data(dataset, n):
    idxs = np.random.permutation(len(dataset.images))
    idxs = idxs[0:n]
    data = torch.tensor(dataset.images[idxs])
    return data.cuda()

def gdt_estimator(samples, **kwargs):
    bs = kwargs['bs']
    ts = kwargs['ts']
    l_ests = torch.zeros(samples.shape)
    for i in range(samples.size(0)):
        l_ests[i,:,:,:], cimg = utils.estimate_lambda(samples[i,0,:,:], bs, ts)
    return l_ests

def net_estimator(samples, **kwargs):
    net = kwargs['network']
    l_ests = torch.zeros(samples.shape)
    for i in range(samples.size(0)):
        with torch.no_grad():
            l_ests[i,:,:,:] = net.process(samples[i,:,:,:].unsqueeze(0).cuda())
    return l_ests

def stupid_estimator(samples, **kwargs):
    l_ests = torch.zeros(samples.shape)
    for i in range(samples.size(0)):
        l_ests[i,:,:,:] = torch.tensor(samples[i,:,:,:])
    return l_ests

def naive_estimator(samples, **kwargs):
    blur = kwargs['blur']
    l_ests = torch.zeros(samples.shape)
    for i in range(samples.size(0)):
        l_ests[i,:,:,:] = utils.imgaussfilt(samples[i,0,:,:], blur).unsqueeze(0).unsqueeze(0)
    return l_ests

def iterated_test(data, n_iters, estimator_function, **kwargs):
    msecrit = torch.nn.MSELoss()
    rdata = data.clone().detach()
    error_hist = list()
    l_est_hist = list()
    rdata_hist = list()
    for i in range(n_iters):
        l_est = estimator_function(rdata, **kwargs)
        l_est_hist.append(l_est.cpu().detach())
        loss = msecrit(l_est.cuda(), data.cuda())
        error_hist.append(loss.item())
        rdata = torch.poisson(l_est.detach()).cpu()
        rdata_hist.append(rdata)
        print('   ', i)
    results = {}
    results['data'] = data
    results['error_hist'] = error_hist
    results['l_est_hist'] = l_est_hist
    results['rdata_hist'] = rdata_hist
    return results

def iterated_test_synthetic(data, n_iters, baseline_estimator, estimator_function, **kwargs):
    msecrit = torch.nn.MSELoss()
    baseline = net_estimator(data, network=baseline_estimator) # synthetic data
    rdata = torch.poisson(baseline) # simulating a MIBI run
    error_hist = list()
    l_est_hist = list()
    rdata_hist = list()
    for i in range(n_iters):
        rdata_hist.append(rdata) # save the 'MIBI' data
        l_est = estimator_function(rdata, **kwargs) # estimate lambda
        l_est_hist.append(l_est.cpu().detach()) # save the estimate
        loss = msecrit(l_est.cuda(), data.cuda()) # get the loss
        error_hist.append(loss.item()) # save the loss
        rdata = torch.poisson(l_est.detach()).cpu() # simulate a MIBI run
        print('   ', i)
    results = {}
    results['data'] = data
    results['error_hist'] = error_hist
    results['l_est_hist'] = l_est_hist
    results['rdata_hist'] = rdata_hist
    return results
    
import re

def get_label(string):
    parts = re.split('_|\.', string)
    return parts[1]

label_dict = {}
for i in range(len(test_ds.source)):
    label = get_label(test_ds.source[i])
    if label in label_dict:
        label_dict[label].append(i)
    else:
        label_dict[label] = [i]

    
    
bs = [10];
while bs[-1]>.7:
    bs.append(bs[-1]*0.8)
ts = torch.tensor(bs).float()**-2
bs = np.array(bs)
ts = ts.numpy()

estimator.eval()
estimator.cuda()
data = get_data_even(test_ds, label_dict).cpu()
n_iters = 20

print('running net_estimator...')
net_estimator_results = iterated_test_synthetic(data, n_iters, estimator, net_estimator, network=estimator)
print()
print('running gdt_estimator...')
gdt_estimator_results = iterated_test_synthetic(data, n_iters, estimator, gdt_estimator, bs=bs, ts=ts)
print()
print('running stupid_estimator...')
stupid_estimator_results = iterated_test_synthetic(data, n_iters, estimator, stupid_estimator)
print()
print('running naive_estimator with blur=1...')
naive_estimator_1_results = iterated_test_synthetic(data, n_iters, estimator, naive_estimator, blur=1)
print()
print('running naive_estimator with blur=5...')
naive_estimator_5_results = iterated_test_synthetic(data, n_iters, estimator, naive_estimator, blur=5)
print()
print('running naive_estimator with blur=20...')
naive_estimator_20_results = iterated_test_synthetic(data, n_iters, estimator, naive_estimator, blur=20)
print()
print('Done')

In [ ]:
# import pickle

# def jar_data(folder, file, data):
#     with open(folder + file, 'wb') as f:
#         pickle.dump(data, f)

# save_folder = '/home/hazmat/GitHub/Denoisotron/data/'
# jar_data(save_folder, 'synthetic_net_estimator_results', net_estimator_results)
# jar_data(save_folder, 'synthetic_gdt_estimator_results', gdt_estimator_results)
# jar_data(save_folder, 'synthetic_stupid_estimator_results', stupid_estimator_results)
# jar_data(save_folder, 'synthetic_naive_estimator_1_results', naive_estimator_1_results)
# jar_data(save_folder, 'synthetic_naive_estimator_5_results', naive_estimator_5_results)
# jar_data(save_folder, 'synthetic_naive_estimator_20_results', naive_estimator_20_results)

In [ ]:
def get_hist2d(x_data, y_data, edges):
    h, xedges, yedges = np.histogram2d(x_data, y_data, bins=edges)
    h = h.T
    sum_val = torch.tensor(np.sum(h,0)).unsqueeze(0).repeat(len(edges)-1,1)
    h = torch.tensor(h)/sum_val
    h = np.nan_to_num(h.numpy(), 0)
    return h, xedges, yedges


def plot_error_hist(network, data, **kwargs):
    fig = plt.figure(figsize=(10,5))
    l = net_estimator(data, network=network)
    smibi_data = torch.poisson(l).cpu()
    l_hat = net_estimator(smibi_data, network=network)
    
    error = torch.abs(l_hat-l)
    data_min = torch.min(l)
    data_max = torch.max(l)
    error_min = torch.min(error)
    error_max = torch.max(error)
    
    xedges = np.linspace(data_min, data_max, 100)
    yedges = np.linspace(error_min, error_max, 100)
    x_data = l.flatten().numpy()
    y_data = error.flatten().numpy()
    
    h, x_edges, y_edges = get_hist2d(x_data, y_data, [xedges, yedges])
    
    ax1 = plt.subplot(1,2,1)
    plt.imshow(l[0,0,:,:])
    plt.axis('off')
    ax2 = plt.subplot(1,2,2)
    img = plt.imshow(h, origin='low', extent=[x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]], **kwargs)
    plt.plot([0,x_edges[-1]], [0,x_edges[-1]], color='red')
    plt.plot([0,x_edges[-1]], [0,-x_edges[-1]], color='red')
    ax2.set_aspect(x_edges[-1]/y_edges[-1])
    plt.xlim(x_edges[0], x_edges[-1])
    plt.ylim(y_edges[0], y_edges[-1])
    fig.colorbar(img, ax=ax2)


def plot_2_error_hist(baseline_estimator, baseline_estimator_args, data, estimator_1, args_1, estimator_2, args_2, **kwargs):
    l = baseline_estimator(data, **baseline_estimator_args)
    x_data = l.flatten().numpy()
    smibi_data = torch.poisson(l).cpu()
    
    l_hat_1 = estimator_1(smibi_data, **args_1)
    error_1 = torch.abs(l_hat_1 - l)
    y_data_1 = error_1.flatten().numpy()
    error_1_min = torch.min(error_1)
    error_1_max = torch.max(error_1)
    
    l_hat_2 = estimator_2(smibi_data, **args_2)
    error_2 = torch.abs(l_hat_2 - l)
    y_data_2 = error_2.flatten().numpy()
    error_2_min = torch.min(error_2)
    error_2_max = torch.max(error_2)
    
    data_min = torch.min(l)
    data_max = torch.max(l)
    error_min = torch.min(error_1_min, error_2_min)
    error_max = torch.max(error_1_max, error_2_max)
    
    xedges = np.linspace(data_min, data_max, 100)
    yedges = np.linspace(error_min, error_max, 100)
    
    h1, _, _ = get_hist2d(x_data, y_data_1, [xedges, yedges])
    h2, _, _ = get_hist2d(x_data, y_data_2, [xedges, yedges])
    
    fig = plt.figure(figsize=(12,8))
    
    ax1 = plt.subplot(2,3,1)
    img1 = plt.imshow(l[0,0,:,:])
    plt.axis('off')
    plt.title('The Truth (is out there)')
    
    ax2 = plt.subplot(2,3,2, sharex=ax1, sharey=ax1)
    img2 = plt.imshow(l_hat_1[0,0,:,:])
    plt.axis('off')
    plt.title(estimator_1.__name__ + ' l-estimate')
    
    ax3 = plt.subplot(2,3,3, sharex=ax1, sharey=ax1)
    img3 = plt.imshow(l_hat_2[0,0,:,:])
    plt.axis('off')
    plt.title(estimator_2.__name__ + ' l-estimate')
    
    ax4 = plt.subplot(2,3,4, sharex=ax1, sharey=ax1)
    img4 = plt.imshow(smibi_data[0,0,:,:])
    plt.axis('off')
    plt.title('"MIBI" data')
    
    ax5 = plt.subplot(2,3,5)
    img5 = plt.imshow(h1, origin='low', extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], cmap='inferno', **kwargs)
    plt.plot([0,xedges[-1]], [0,xedges[-1]], color='green')
    plt.plot([0,xedges[-1]], [0,-xedges[-1]], color='green')
    ax5.set_aspect(xedges[-1]/yedges[-1])
    plt.xlim(xedges[0], xedges[-1])
    plt.ylim(yedges[0], yedges[-1])
    plt.title(estimator_1.__name__ + ' error dist.')
    
    ax6 = plt.subplot(2,3,6, sharex=ax5, sharey=ax5)
    img5 = plt.imshow(h2, origin='low', extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], cmap='inferno', **kwargs)
    plt.plot([0,xedges[-1]], [0,xedges[-1]], color='green')
    plt.plot([0,xedges[-1]], [0,-xedges[-1]], color='green')
    ax6.set_aspect(xedges[-1]/yedges[-1])
    plt.xlim(xedges[0], xedges[-1])
    plt.ylim(yedges[0], yedges[-1])
    plt.title(estimator_2.__name__ + ' error dist.')
    # fig.colorbar(img2, ax=ax3)
    
    return [img1, img2, img3, img4]

def update_clims(imgs, cmax):
    for img in imgs:
        img.set_clim(0, cmax)

In [ ]:
imgs = plot_2_error_hist(gdt_estimator, {'bs': bs, 'ts': ts}, test_ds.images[6].unsqueeze(0), net_estimator, {'network': estimator}, gdt_estimator, {'bs': bs, 'ts': ts}, vmax=.1)
# imgs = plot_2_error_hist(net_estimator, {'network': estimator}, test_ds.images[6].unsqueeze(0), net_estimator, {'network': estimator}, naive_estimator, {'blur': 1}, vmax=.2)

In [ ]:
update_clims(imgs, .5)

In [ ]:
iter_fig = plt.figure(figsize=(7,5))
# plt.plot(stupid_estimator_results['error_hist'], color=[1,0,1], label='stupid_estimator')
plt.plot(naive_estimator_1_results['error_hist'], color=[1,0,0], label='naive_estimator_1')
# plt.plot(naive_estimator_5_results['error_hist'], color=[1,.5,0], label='naive_estimator_5')
# plt.plot(naive_estimator_20_results['error_hist'], color=[1,1,0], label='naive_estimator_20')
plt.plot(gdt_estimator_results['error_hist'], color=[0,0,1], label='gdt_estimator')
plt.plot(net_estimator_results['error_hist'], color=[0,1,0], label='net_estimator')
plt.legend()
plt.xticks(range(0,20,4))
plt.xlim(0,19)
plt.xlabel('Iteration')
plt.ylabel('MSE')

In [ ]:
global data
global max_abs_val
global idx

idx = 4
data = gdt_estimator_results
max_abs_val = 0.0

for i in range(20):
    sub_maxval = torch.max(torch.abs(data['l_est_hist'][i][idx,:,:,:]-data['l_est_hist'][0][idx,:,:,:])).item()
    max_abs_val = np.max([max_abs_val, sub_maxval])
    
print(max_abs_val)

In [ ]:
from ipywidgets import *

diff_fig = plt.figure(figsize=(15,5))
# max_abs_val = 2


def update(index=0):
    global data
    global max_abs_val
    global idx
    
    ax1 = plt.subplot(1,3,1)
    plt.imshow(data['l_est_hist'][0][idx,0,:,:])
    plt.axis('off')
    
    diff = data['l_est_hist'][index]-data['l_est_hist'][0]
    ax2 = plt.subplot(1,3,2, sharex=ax1, sharey=ax1)
    plt.imshow(diff[idx,0,:,:], vmin=-max_abs_val, vmax=max_abs_val)
    plt.axis('off')
    plt.title('Error on Iteration ' + str(index))
    
    ax3 = plt.subplot(1,3,3, sharex=ax1, sharey=ax1)
    plt.imshow(data['l_est_hist'][index][idx,0,:,:])
    plt.axis('off')
    
interact(update, index=widgets.IntSlider(value=0, min=0, max=19, step=1))

In [ ]:
comp_fig = plt.figure(figsize=(12,6))
gdt_est = gdt_estimator_results['l_est_hist'][0][idx,0,:,:]
net_est = net_estimator_results['l_est_hist'][0][idx,0,:,:]
ax1 = plt.subplot(1,2,1)
plt.imshow(gdt_est)
ax2 = plt.subplot(1,2,2, sharex=ax1, sharey=ax1)
plt.imshow(net_est)

In [ ]:
data_path = '/home/hazmat/GitHub/Denoisotron/data/shirley_point/TIFs/CD45.tif'
data = io.imread(data_path).astype(int)
data = torch.tensor(data, dtype=torch.uint8).float()

In [ ]:
bs = [10];
while bs[-1]>.7:
    bs.append(bs[-1]*0.8)

ts = torch.tensor(bs).float()**-2
bs = np.array(bs)
ts = ts.numpy()
print(bs)
print(ts)

In [ ]:
lambda_est, c_img = utils.estimate_lambda(data, bs, ts)

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(lambda_est)